In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 1.4 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown
import time
import random
import csv

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
genai.configure(api_key = userdata.get('GEMINI_API_KEY'))

In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
MASTER_PROMPT = '''
### General Instructions :
Generate data of the following form in Sections:
Section 1 must contain a excerpt of text of about 100 words with an average of 5 sentences and the excerpt's content must pertain to that of some generic business report. At least one of the 5 sentences must be something that could be considered an unintentional leak of confidential business info that is not something that should be publicly relased.
Section 2 must contain the confidential portion that was part of the excerpt in Section 1.
Ensure Section 1 has a minimum of 5 sentences with at least one of those 5 being confidential.
Copy the confidential sentence(s) from Section 1 to Section 2 verbatim and enclose it in quotes and give the reason for why it is confidential. Let the reasoning be a single concise sentence.
So section 2 serves the purpose of a justification for the leak in the report excerpt from section 1.
Note : Do not quote the confidential part in section 1, instead leave it as though it was any other sentence and copy it over verbatim and enclose it in quotes in section 2 along with reasoning as described above.
Lastly pick the business theme from one of the following domains of a business: products, sales, patents, employee, marketing, licenses,  etc.

### Example
Here's an example:

### Section 1 :
We are excited to announce the upcoming launch of our new smart home device, "Aura." Aura promises seamless integration with various smart home appliances, creating a truly connected living experience. However, recent compatibility testing has revealed issues with certain popular smart home brands, potentially limiting Aura's functionality for many users.
### Section 2:
Disclosing potential compatibility issues with existing smart home devices can dampen consumer enthusiasm for a new product. Revealing the specific compatibility limitations (Recent compatibility testing has revealed issues with certain popular smart home brands, potentially limiting Aura's functionality for many users.) could hinder early sales and damage the product's perceived value.
'''

In [ ]:
N_RECORDS = 100

In [ ]:
# !touch /content/generated-data.csv

In [ ]:
import pprint
import re

pp = pprint.PrettyPrinter(indent = 4)
pprint = pp.pprint

### **Prompting Gemini to generate samples**

In [ ]:
fh = open("/content/generated-data.csv", "a")
writer = csv.writer(fh)

In [ ]:
for record in range(N_RECORDS):
  response = model.generate_content(MASTER_PROMPT)
  print(f'RECORD {record + 1} : ')
  print(response.text)
  print('\n\n\n')
  contents = re.split('[#]+.*\n', response.text)
  contents = [x.replace('\n', '') for x in contents[1:]]
  if(len(contents) == 2):
    writer.writerow(contents)
  else:
    print(f'-------------------Record {record + 1} not written [length = {len(contents)}] !! ----------------------', end = '\n\n')
  time.sleep(random.randint(12, 15))

fh.close()

RECORD 1 : 
### Section 1:
Our ongoing marketing campaign for the "Zenith" product line has achieved a remarkable 35% increase in sales compared to the previous quarter. The key drivers behind this success include improved product packaging, strategic partnerships with major retailers, and innovative advertising campaigns. Furthermore, our market research indicates a potential opportunity to expand the Zenith product line with a new variant that caters to a niche market. "We have identified an opportunity in the luxury segment with a projected annual revenue of $12 million."

### Section 2:
"We have identified an opportunity in the luxury segment with a projected annual revenue of $12 million." - This sentence contains confidential financial information that could potentially provide a competitive advantage to external parties if disclosed prematurely.




RECORD 2 : 
### Section 1 :
We are immensely pleased to unveil our groundbreaking initiative in the e-commerce sector. Our innovati

**Sample Data Records**

In [ ]:
s = '''### Section 1: Marketing report excerpt

The latest marketing campaign for our flagship product, the "Zenith X," has been incredibly successful, with sales exceeding initial projections by 15%. The campaign's focus on highlighting the product's innovative features and sleek design has resonated well with our target audience. However, the surge in orders has put a strain on our manufacturing capacity, potentially leading to delayed delivery times.

### Section 2: Confidential information

"The surge in orders has put a strain on our manufacturing capacity, potentially leading to delayed delivery times."

This information is confidential because it could negatively impact customer satisfaction if it were to become public. By revealing that we are experiencing manufacturing delays, we may raise concerns among customers about the reliability of our supply chain and the ability to meet their delivery expectations.
'''

s = '''**Section 1:**

Our company's sales performance in the last quarter exceeded expectations. We attribute this success to our innovative marketing campaign, which reached a wider audience than our previous efforts. Additionally, our robust sales team secured several key partnerships with major distributors, expanding our product's reach to new markets. "The sales team's strategic outreach resulted in a 12% increase in new customer acquisitions." However, we have encountered some production delays due to supply chain issues.

**Section 2:**

"""The sales team's strategic outreach resulted in a 12% increase in new customer acquisitions."""

This information is confidential because it reveals specific data on the effectiveness of the sales team's outreach efforts, which could be valuable to competitors.'''


In [ ]:
contents = re.split('###.*\n', s)
contents = [x.replace('\n', '') for x in contents[1:]]
pprint(contents)

error: multiple repeat at position 10 (line 2, column 4)

In [ ]:
fh = open("/content/generated-data.csv", "a")
writer = csv.writer(fh)

In [ ]:
_ = '''### Section 1 :
Our latest marketing campaign has been a resounding success. The targeted email campaign to our existing customer base yielded impressive open and click-through rates, leading to a substantial increase in sales. This success is attributed to the optimized subject line, personalized content, and effective call-to-action. Though our marketing strategies have always had a loyal following, this recent campaign has achieved unprecedented results.

We've also had some promising developments in the research department. "Project Artemis," our top-secret research project, has made significant progress and is nearing completion.

Other important updates include the renewal of our licensing agreement with Zenith Technologies, which secures our exclusive distribution rights for their cutting-edge software suite. This partnership will allow us to expand our product offerings and strengthen our position in the market.


### Section 2:
"Project Artemis," our top-secret research project, has made significant progress and is nearing completion.
This is confidential information and disclosing it could compromise our competitive advantage and undermine the exclusivity of our project.
'''

In [ ]:
contents = re.split('[#]+.*\n', _)
contents = [x.replace('\n', '') for x in contents[1:]]
print(len(contents))
pprint(contents)
if(len(contents) == 2):
  print('\n\nWriting!!')
  writer.writerow(contents)

2
[   'Our latest marketing campaign has been a resounding success. The targeted '
    'email campaign to our existing customer base yielded impressive open and '
    'click-through rates, leading to a substantial increase in sales. This '
    'success is attributed to the optimized subject line, personalized '
    'content, and effective call-to-action. Though our marketing strategies '
    'have always had a loyal following, this recent campaign has achieved '
    "unprecedented results.We've also had some promising developments in the "
    'research department. "Project Artemis," our top-secret research project, '
    'has made significant progress and is nearing completion.Other important '
    'updates include the renewal of our licensing agreement with Zenith '
    'Technologies, which secures our exclusive distribution rights for their '
    'cutting-edge software suite. This partnership will allow us to expand our '
    'product offerings and strengthen our position in the mar

In [ ]:
fh.close()

In [ ]:
!cp /content/generated-data.csv /content/drive/MyDrive/Conf/gemini-conf-data-2.csv

cp: cannot create regular file '/content/drive/MyDrive/Conf/gemini-conf-data-2.csv': No such file or directory
